In [1]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/usr/bin/python3'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2.3.1.0.0-78
      /_/

Using Python version 2.7.16 (default, Sep 24 2019 21:51:30)
SparkSession available as 'spark'.


In [2]:
linux_auth_logs = 'linux_auth_logs/auth.log*'
df_raw = spark.read.text(linux_auth_logs)

In [3]:
df_raw.show(3,False)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|Oct  6 06:25:15 de-5-maria-sokolova-cn1 CRON[26658]: pam_unix(cron:session): session closed for user root|
|Oct  6 06:27:42 de-5-maria-sokolova-cn1 su[14001]: pam_unix(su:session): session closed for user root    |
|Oct  6 06:27:42 de-5-maria-sokolova-cn1 sudo: pam_unix(sudo:session): session closed for user root       |
+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [4]:
from pyspark.sql import functions as sf

In [5]:
log_line_dt = sf.split(df_raw["value"],"de-5-maria-sokolova-cn1")

In [6]:
log_line_hostame_event = sf.split( sf.substring(df_raw["value"],16,2048)," ")

In [7]:
df_raw.withColumn("hostname", log_line_hostame_event[2]).show(5,False)

+---------------------------------------------------------------------------------------------------------+------------+
|value                                                                                                    |hostname    |
+---------------------------------------------------------------------------------------------------------+------------+
|Oct  6 06:25:15 de-5-maria-sokolova-cn1 CRON[26658]: pam_unix(cron:session): session closed for user root|CRON[26658]:|
|Oct  6 06:27:42 de-5-maria-sokolova-cn1 su[14001]: pam_unix(su:session): session closed for user root    |su[14001]:  |
|Oct  6 06:27:42 de-5-maria-sokolova-cn1 sudo: pam_unix(sudo:session): session closed for user root       |sudo:       |
|Oct  6 06:27:43 de-5-maria-sokolova-cn1 su[28021]: Successful su for ambari-qa by root                   |su[28021]:  |
|Oct  6 06:27:43 de-5-maria-sokolova-cn1 su[28021]: + ??? root:ambari-qa                                  |su[28021]:  |
+-------------------------------

In [8]:
log_message = sf.split(df_raw["value"],"]: ")

In [9]:
df_raw=df_raw.withColumn("dt", log_line_dt[0])
df_raw=df_raw.withColumn("hostname", log_line_hostame_event[1])
df_raw=df_raw.withColumn("event", log_line_hostame_event[2])
df_raw=df_raw.withColumn("message", log_message[1])

In [10]:
df_raw.select(["dt","hostname","event","message"]).show(10,False)

+----------------+-----------------------+--------------------+------------------------------------------------------------------+
|dt              |hostname               |event               |message                                                           |
+----------------+-----------------------+--------------------+------------------------------------------------------------------+
|Oct  6 06:25:15 |de-5-maria-sokolova-cn1|CRON[26658]:        |pam_unix(cron:session): session closed for user root              |
|Oct  6 06:27:42 |de-5-maria-sokolova-cn1|su[14001]:          |pam_unix(su:session): session closed for user root                |
|Oct  6 06:27:42 |de-5-maria-sokolova-cn1|sudo:               |null                                                              |
|Oct  6 06:27:43 |de-5-maria-sokolova-cn1|su[28021]:          |Successful su for ambari-qa by root                               |
|Oct  6 06:27:43 |de-5-maria-sokolova-cn1|su[28021]:          |+ ??? root:ambari-qa

In [11]:
df_raw = df_raw.withColumn("datetime", sf.concat(sf.lit("2019 "),df_raw["dt"])) #.show(5)

In [12]:
df_raw.show(3, False)

+---------------------------------------------------------------------------------------------------------+----------------+-----------------------+------------+----------------------------------------------------+---------------------+
|value                                                                                                    |dt              |hostname               |event       |message                                             |datetime             |
+---------------------------------------------------------------------------------------------------------+----------------+-----------------------+------------+----------------------------------------------------+---------------------+
|Oct  6 06:25:15 de-5-maria-sokolova-cn1 CRON[26658]: pam_unix(cron:session): session closed for user root|Oct  6 06:25:15 |de-5-maria-sokolova-cn1|CRON[26658]:|pam_unix(cron:session): session closed for user root|2019 Oct  6 06:25:15 |
|Oct  6 06:27:42 de-5-maria-sokolova-cn1 su[14001]: 

In [13]:
datetime_format = "yyyy MMM d HH:mm:ss" #2019 Oct  2 06:25:12

In [14]:
datetime_to_date = sf.to_timestamp(df_raw["datetime"],datetime_format)

In [15]:
df_raw = df_raw.withColumn("timestamp", datetime_to_date)

In [16]:
event_pid = sf.regexp_extract(df_raw["event"],"\d+",0)

In [17]:
event_type = sf.regexp_extract(df_raw["event"],"[a-zA-Z]+",0)

In [18]:
df_raw = df_raw.withColumn("event_pid", event_pid)

In [19]:
df_raw = df_raw.withColumn("event_type", event_type)

In [20]:
df = df_raw[("timestamp","hostname","event","event_type","event_pid","message")]

http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=substring#pyspark.sql.functions.regexp_extract

In [21]:
maxtimestamp = df.agg({"timestamp":"max"})

In [22]:
maxtimestamp_col = maxtimestamp['max(timestamp)']

In [23]:
maxtimestamp_row = maxtimestamp.first()

In [24]:
maxtimestamp_dict = maxtimestamp_row.asDict()

In [25]:
maxtimestamp_value = maxtimestamp_dict.get('max(timestamp)')

In [26]:
from datetime import timedelta 

In [27]:
print(maxtimestamp_value)

2019-10-09 07:56:52


In [28]:
maxtimestamp_value + timedelta(minutes=10)

datetime.datetime(2019, 10, 9, 8, 6, 52)

In [29]:
df.filter("event_pid='4896'").show(200, False)

+-------------------+-----------------------+-----------+----------+---------+------------------------------------------------------------------------------------------------------------------------+
|timestamp          |hostname               |event      |event_type|event_pid|message                                                                                                                 |
+-------------------+-----------------------+-----------+----------+---------+------------------------------------------------------------------------------------------------------------------------+
|2019-10-07 04:14:05|de-5-maria-sokolova-cn1|sshd[4896]:|sshd      |4896     |Accepted publickey for ubuntu from 176.193.61.84 port 37436 ssh2: RSA SHA256:PhgwaDhr3B+un+nGF4UwoKNwlxAuWfiP4+LgDhXprV0|
|2019-10-07 04:14:05|de-5-maria-sokolova-cn1|sshd[4896]:|sshd      |4896     |pam_unix(sshd:session): session opened for user ubuntu by (uid=0)                                                       |


In [30]:
df

DataFrame[timestamp: timestamp, hostname: string, event: string, event_type: string, event_pid: string, message: string]

In [31]:
pd_df = df.toPandas()

In [32]:
pd_df.head(3)

,timestamp,hostname,event,event_type,event_pid,message
0,2019-10-06 06:25:15,de-5-maria-sokolova-cn1,CRON[26658]:,CRON,26658,pam_unix(cron:session): session closed for use...
1,2019-10-06 06:27:42,de-5-maria-sokolova-cn1,su[14001]:,su,14001,pam_unix(su:session): session closed for user ...
2,2019-10-06 06:27:42,de-5-maria-sokolova-cn1,sudo:,sudo,,None


In [33]:
import pandas as pd

In [34]:
#df.timestamp.dt.to_period(freq="10Min")
pd_df["ts"] = pd.DatetimeIndex(pd_df.timestamp)

In [35]:
pd_df["ts_start"]=pd_df.ts.dt.floor("10min")

In [36]:
pd_df.head()

,timestamp,hostname,event,event_type,event_pid,message,ts,ts_start
0,2019-10-06 06:25:15,de-5-maria-sokolova-cn1,CRON[26658]:,CRON,26658,pam_unix(cron:session): session closed for use...,2019-10-06 06:25:15,2019-10-06 06:20:00
1,2019-10-06 06:27:42,de-5-maria-sokolova-cn1,su[14001]:,su,14001,pam_unix(su:session): session closed for user ...,2019-10-06 06:27:42,2019-10-06 06:20:00
2,2019-10-06 06:27:42,de-5-maria-sokolova-cn1,sudo:,sudo,,None,2019-10-06 06:27:42,2019-10-06 06:20:00
3,2019-10-06 06:27:43,de-5-maria-sokolova-cn1,su[28021]:,su,28021,Successful su for ambari-qa by root,2019-10-06 06:27:43,2019-10-06 06:20:00
4,2019-10-06 06:27:43,de-5-maria-sokolova-cn1,su[28021]:,su,28021,+ ??? root:ambari-qa,2019-10-06 06:27:43,2019-10-06 06:20:00


In [39]:
from datetime import timedelta

In [40]:
from datetime import datetime

In [42]:
datetime.now() + timedelta(0,0,0,0,10)

datetime.datetime(2019, 10, 11, 6, 30, 40, 580917)

In [45]:
df.select(sf.to_timestamp(sf.concat(sf.substring(sf.expr("cast(timestamp as string)"),1,15) , sf.lit("0:00"))) ).show(20)

+---------------------------------------------------------------------------+
|to_timestamp(concat(substring(CAST(`timestamp` AS STRING), 1, 15), '9:59'))|
+---------------------------------------------------------------------------+
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10-06 06:29:59|
|                                                        2019-10

In [46]:
start_ts = sf.to_timestamp(sf.concat(sf.substring(sf.expr("cast(timestamp as string)"),1,15) , sf.lit("0:00"))) 

In [47]:
df = df.withColumn("start_ts",start_ts)

In [48]:
df.show(20)

+-------------------+--------------------+--------------------+----------+---------+--------------------+-------------------+
|          timestamp|            hostname|               event|event_type|event_pid|             message|           start_ts|
+-------------------+--------------------+--------------------+----------+---------+--------------------+-------------------+
|2019-10-06 06:25:15|de-5-maria-sokolo...|        CRON[26658]:|      CRON|    26658|pam_unix(cron:ses...|2019-10-06 06:20:00|
|2019-10-06 06:27:42|de-5-maria-sokolo...|          su[14001]:|        su|    14001|pam_unix(su:sessi...|2019-10-06 06:20:00|
|2019-10-06 06:27:42|de-5-maria-sokolo...|               sudo:|      sudo|         |                null|2019-10-06 06:20:00|
|2019-10-06 06:27:43|de-5-maria-sokolo...|          su[28021]:|        su|    28021|Successful su for...|2019-10-06 06:20:00|
|2019-10-06 06:27:43|de-5-maria-sokolo...|          su[28021]:|        su|    28021|+ ??? root:ambari-qa|2019-10-06 06

In [52]:
from pyspark.sql import Window

In [53]:
window = Window.partitionBy("start_ts")

In [55]:
agg = df.withColumn("row_count", sf.count("*").over(window))
   

In [65]:
df.groupBy("start_ts").agg(sf.count("*").alias("event_count"), sf.max('event_pid').alias("pid_max")).orderBy("start_ts").show(20) #, sum('pid').alias("pid_sum"))

+-------------------+-----------+-------+
|           start_ts|event_count|pid_max|
+-------------------+-----------+-------+
|2019-09-24 18:20:00|          6|    988|
|2019-09-24 18:30:00|          8|    851|
|2019-09-24 18:40:00|          2|    988|
|2019-09-25 03:50:00|          9|    961|
|2019-09-25 04:00:00|         27|    921|
|2019-09-25 04:10:00|         14|    921|
|2019-09-25 04:20:00|         36|    961|
|2019-09-25 04:30:00|         59|    961|
|2019-09-25 04:40:00|         24|    943|
|2019-09-25 04:50:00|         65|    983|
|2019-09-25 05:00:00|         50|    984|
|2019-09-25 05:10:00|         26|    895|
|2019-09-25 05:20:00|         23|   5892|
|2019-09-25 05:30:00|          3|   6042|
|2019-09-25 05:40:00|         23|   6420|
|2019-09-25 05:50:00|        146|    895|
|2019-09-25 06:00:00|         18|    895|
|2019-09-25 06:10:00|        143|   9963|
|2019-09-25 06:20:00|          6|  12324|
|2019-09-25 06:30:00|         79|  13910|
+-------------------+-----------+-

In [81]:
df.groupBy("start_ts").agg(sf.countDistinct("event_type").alias("event_type_count")).\
join(df.filter("event_type='sshd'").groupBy("start_ts").agg(sf.count("*").alias("sshd_event_count")),"start_ts","left").show(20)

+-------------------+----------------+----------------+
|           start_ts|event_type_count|sshd_event_count|
+-------------------+----------------+----------------+
|2019-10-06 09:10:00|               4|               4|
|2019-10-01 10:30:00|               5|               2|
|2019-10-06 09:50:00|               4|               2|
|2019-10-08 12:40:00|               4|               8|
|2019-10-06 09:00:00|               3|            null|
|2019-09-26 12:20:00|               4|               2|
|2019-10-01 11:30:00|               3|            null|
|2019-10-03 14:30:00|               3|            null|
|2019-10-04 13:00:00|               4|               4|
|2019-10-06 06:50:00|               4|               2|
|2019-10-04 08:10:00|               4|               4|
|2019-10-07 08:10:00|               5|               4|
|2019-10-07 17:40:00|               3|            null|
|2019-10-08 14:50:00|               4|               1|
|2019-10-04 06:10:00|               4|          

In [87]:
df.groupBy("start_ts").agg(sf.countIf("event_type='sshd'"))

AttributeError: 'module' object has no attribute 'countIf'